In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
!pip install pdfminer.six python-docx pandas numpy scikit-learn spacy streamlit transformers
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 129.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("snehaanbhawal/resume-dataset")

100%|██████████| 62.5M/62.5M [00:00<00:00, 134MB/s]

Extracting files...


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/resume_matching/resume_data.csv')

In [ ]:
df.head(10)

,address,career_objective,skills,educational_institution_name,degree_names,passing_years,educational_results,result_types,major_field_of_studies,professional_company_names,...,online_links,issue_dates,expiry_dates,﻿job_position_name,educationaL_requirements,experiencere_requirement,age_requirement,responsibilities.1,skills_required,matched_score
0,NaN,Big data analytics working and database wareho...,"['Big Data', 'Hadoop', 'Hive', 'Python', 'Mapr...",['The Amity School of Engineering & Technology...,['B.Tech'],['2019'],['N/A'],[None],['Electronics'],['Coca-COla'],...,NaN,NaN,NaN,Senior Software Engineer,B.Sc in Computer Science & Engineering from a ...,At least 1 year,NaN,Technical Support\nTroubleshooting\nCollaborat...,NaN,0.850000
1,NaN,Fresher looking to join as a data analyst and ...,"['Data Analysis', 'Data Analytics', 'Business ...","['Delhi University - Hansraj College', 'Delhi ...","['B.Sc (Maths)', 'M.Sc (Science) (Statistics)']","['2015', '2018']","['N/A', 'N/A']","['N/A', 'N/A']","['Mathematics', 'Statistics']",['BIB Consultancy'],...,NaN,NaN,NaN,Machine Learning (ML) Engineer,M.Sc in Computer Science & Engineering or in a...,At least 5 year(s),NaN,Machine Learning Leadership\nCross-Functional ...,NaN,0.750000
2,NaN,NaN,"['Software Development', 'Machine Learning', '...","['Birla Institute of Technology (BIT), Ranchi']",['B.Tech'],['2018'],['N/A'],['N/A'],['Electronics/Telecommunication'],['Axis Bank Limited'],...,NaN,NaN,NaN,"Executive/ Senior Executive- Trade Marketing, ...",Master of Business Administration (MBA),At least 3 years,NaN,"Trade Marketing Executive\nBrand Visibility, S...",Brand Promotion\nCampaign Management\nField Su...,0.416667
3,NaN,To obtain a position in a fast-paced business ...,"['accounts payables', 'accounts receivables', ...","['Martinez Adult Education, Business Training ...",['Computer Applications Specialist Certificate...,['2008'],[None],[None],['Computer Applications'],"['Company Name ï¼ City , State', 'Company Name...",...,NaN,NaN,NaN,Business Development Executive,Bachelor/Honors,1 to 3 years,Age 22 to 30 years,Apparel Sourcing\nQuality Garment Sourcing\nRe...,Fast typing skill\nIELTSInternet browsing & on...,0.760000
4,NaN,Professional accountant with an outstanding wo...,"['Analytical reasoning', 'Compliance testing k...",['Kent State University'],['Bachelor of Business Administration'],[None],['3.84'],[None],['Accounting'],"['Company Name', 'Company Name', 'Company Name...",...,[None],[None],"['February 15, 2021']",Senior iOS Engineer,Bachelor of Science (BSc) in Computer Science,At least 4 years,NaN,iOS Lifecycle\nRequirement Analysis\nNative Fr...,iOS\niOS App Developer\niOS Application Develo...,0.650000
5,NaN,"To secure an IT specialist, desktop support, n...","['Microsoft Applications', 'Network Security',...","['Glen Oaks High School', 'Glen Oaks High Scho...","['Bachelor Degree', 'Associate Degree']","[None, None]","[None, None]","[None, None]",['Electronics and Communications Engineering T...,"['N/A', 'Company Name', 'Company Name']",...,[None],['May 2013'],[None],AI Engineer,Bachelors or Masters degree in Computer Scienc...,NaN,NaN,Machine Learning Design\nData Analysis\nModel ...,Python\nR or Java\nTensorFlow\nPyTorch\nScikit...,0.850000
6,NaN,NaN,"['Machine Learning', 'Linear Regression', 'Rid...",['DJR College and University'],['B.Tech'],['2020'],['N/A'],['N/A'],['IT'],['Remiro Amio'],...,NaN,NaN,NaN,Senior iOS Engineer,Bachelor of Science (BSc) in Computer Science,At least 4 years,NaN,iOS Lifecycle\nRequirement Analysis\nNative Fr...,iOS\niOS App Developer\niOS Application Develo...,0.650000
7,NaN,NaN,"['Maintenance', 'Corrective Maintenance', 'Doc...",['POLYTECHNIC UNIVERSITY OF PUERTO RICO'],['Bachelor of Science'],['2009'],['2.50'],['GPA'],['Electrical Engineering'],"['Company Name', 'Company Name', 'Company Name']",...,[None],[None],[None],Senior iOS Engineer,Bachelor of Science (BSc) in Computer Science,At least 4 years,NaN,iOS Lifecycle\nRequirement Analysis\n

In [ ]:
df.columns

Index(['address', 'career_objective', 'skills', 'educational_institution_name',
       'degree_names', 'passing_years', 'educational_results', 'result_types',
       'major_field_of_studies', 'professional_company_names', 'company_urls',
       'start_dates', 'end_dates', 'related_skils_in_job', 'positions',
       'locations', 'responsibilities', 'extra_curricular_activity_types',
       'extra_curricular_organization_names',
       'extra_curricular_organization_links', 'role_positions', 'languages',
       'proficiency_levels', 'certification_providers', 'certification_skills',
       'online_links', 'issue_dates', 'expiry_dates', '﻿job_position_name',
       'educationaL_requirements', 'experiencere_requirement',
       'age_requirement', 'responsibilities.1', 'skills_required',
       'matched_score'],
      dtype='object')

In [ ]:
text_columns = [
    'career_objective', 'skills', 'related_skils_in_job',
    'positions', 'responsibilities','skills_required', 'educationaL_requirements', 'responsibilities.1'
]

In [ ]:
def combine_text(row):
    return " ".join([str(row[col]) for col in text_columns if pd.notnull(row[col])])

df['combined_text'] = df.apply(combine_text, axis=1)

In [ ]:
df.combined_text

,combined_text
0,Big data analytics working and database wareho...
1,Fresher looking to join as a data analyst and ...
2,"['Software Development', 'Machine Learning', '..."
3,To obtain a position in a fast-paced business ...
4,Professional accountant with an outstanding wo...
...,...
9539,"['Mathematical modelling', 'Machine Learning',..."
9540,Expertise EDA modeler. I like to learn what my...
9541,Looking for roles related to application devel...
9542,"['Machine Learning', 'Natural Language Process..."


In [9]:
import spacy
nlp = spacy.load("en_core_web_sm")

def preprocess(text):
    doc = nlp(text.lower())
    return " ".join([token.lemma_ for token in doc if token.is_alpha and not token.is_stop])

df['cleaned_text'] = df['combined_text'].apply(preprocess)

In [ ]:
!pip install numpy==1.23.5 --force-reinstall
!pip install --force-reinstall --no-cache-dir gensim

  Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.3 kB)
Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
xarray 2025.3.1 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.23.5 which is incompatible.
bigframes 2.4.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
blosc2 3.3.2 requires numpy>=1.26, 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.5 MB/s eta 0:00:00
ERROR: Operation cancelled by user
^C


In [ ]:
!pip install gensim
import gensim
from gensim.models import Word2Vec
# Tokenize for Word2Vec
df['tokens'] = df['combined_text'].apply(lambda x: x.split())

w2v_model = Word2Vec(sentences=df['tokens'], vector_size=100, window=5, min_count=1)

def get_vector(tokens):
    vec = []
    for token in tokens:
        if token in w2v_model.wv:
            vec.append(w2v_model.wv[token])
    return np.mean(vec, axis=0) if vec else np.zeros(100)

df['w2v_vector'] = df['tokens'].apply(get_vector)
X_w2v = np.vstack(df['w2v_vector'].values)


In [ ]:
# Remove the numpy version constraint
!pip uninstall numpy --yes
# Reinstall the necessary libraries, letting pip handle numpy version
!pip install pdfminer.six python-docx pandas numpy scikit-learn spacy streamlit transformers sentence-transformers
!python -m spacy download en_core_web_sm
!pip install gensim

Found existing installation: numpy 1.23.5
Uninstalling numpy-1.23.5:
  Successfully uninstalled numpy-1.23.5
  Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 13.5 MB/s eta 0:00:00
Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.5 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 102.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
vectorizer = TfidfVectorizer(max_features=1000)

In [13]:
from sklearn.metrics.pairwise import cosine_similarity

job_desc = "We need a backend developer with experience in Python, Flask, and databases."
job_clean = preprocess(job_desc)

In [17]:
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['combined_text'])
job_vec = vectorizer.transform([job_clean]) # tf

In [18]:
similarities = cosine_similarity(job_vec, X).flatten()
df['similarity_score'] = similarities

In [22]:
ranked_resumes = df.sort_values(by='similarity_score', ascending=False).reset_index(drop=True)

top_n = 5
print(f"\nTop {top_n} Candidates Matching the Job Description:\n")
for i, row in ranked_resumes.head(top_n).iterrows():
    print(f"Rank {i+1}:")
    print(f"Skills     : {row['skills']}")
    print(f"Match Score  : {round(row['similarity_score'], 4)}")
    print(f"Resume Snippet: {row.get('Resume', row['cleaned_text'])[:300]}...")
    print("="*85)



Top 5 Candidates Matching the Job Description:

Rank 1:
Skills     : ['Python', 'MySQL', 'Natural Language Processing', 'Machine Learning', 'Neural Networks', 'Flask', 'Docker', 'GIT', 'BERT', 'LSTM', 'RNN', 'Deep Learning', 'AWS', 'Tensorflow', 'Keras', 'PyTorch', 'Numpy', 'Pandas', 'Glove', 'Lucene', 'Linux', 'Devops', 'Spacy', 'GPU', 'Data Science']
Match Score  : 0.2358
Resume Snippet: python mysql natural language processing machine learning neural network flask docker git bert lstm rnn deep learning aw tensorflow keras pytorch numpy panda glove lucene linux devop spacy gpu data science bert gpu docker flask framework dashboard real time monitoring sensor datum machine learn engi...
Rank 2:
Skills     : ['Python Developer', 'Django', 'Flask', 'Data Science', 'Spark', 'PySpark', 'Machine Learning', 'Data Modelling', 'Natural language Processing', 'SVM', 'Computer Vision', 'Neural Networks.']
Match Score  : 0.2352
Resume Snippet: python programmer currently pursue datum science loo

In [47]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

bert_model = SentenceTransformer('all-MiniLM-L6-v2')
X_bert = bert_model.encode(df['combined_text'].tolist())

job_desc = "We need a backend developer with experience in Python, Flask, and databases."
job_clean = preprocess(job_desc)

job_embedding = bert_model.encode([job_clean])
similarities = cosine_similarity(job_embedding, X_bert).flatten()

df['similarity_score'] = similarities

ranked_resumes = df.sort_values(by='similarity_score', ascending=False).reset_index(drop=True)
top_n = 5
print(f"\nTop {top_n} Candidates Matching the Job Description (BERT):\n")
for i, row in ranked_resumes.head(top_n).iterrows():
    print(f"Rank {i+1}:")
    print(f"Skills       : {row.get('skills', 'N/A')}")
    print(f"Match Score  : {round(row['similarity_score'], 4)}")
    print(f"Resume Snippet: {row.get('Resume', row.get('cleaned_text', ''))[:300]}...")
    print("="*85)


Top 5 Candidates Matching the Job Description (BERT):

Rank 1:
Skills       : ['Python', 'LeetCode', 'GDB', 'GitHub', 'HTML', 'MySQL', 'Lucidchart', 'Postman', 'OWASP ZAP', 'Jira']
Match Score  : 0.5894
Resume Snippet: python leetcode gdb github html mysql lucidchart postman owasp zap jira python gdb github powershell owasp zap jira computer programmer system administrator help desk technician stack development end reactjs nextjs backend python django api design server logic drf django rest framework database mana...
Rank 2:
Skills       : ['Python', 'Machine Learning', 'Deep Learning', 'R', 'Computer Vision', 'Numpy', 'Panda', 'Matplotlib', 'MySQL', 'OpenCV', 'PyTorch']
Match Score  : 0.5885
Resume Snippet: python machine learning deep learning r computer vision numpy panda matplotlib mysql opencv pytorch nlp datum scrapping python beautiful soup image processing python satellite imagery intern trainee intern developer stack development end reactjs nextjs backend python django api de

In [30]:
print(df['label'].value_counts())


label
0    9544
Name: count, dtype: int64


In [33]:
df['label'] = (df['similarity_score'] > 0.5).astype(int)
print(df['similarity_score'].describe())


count    9544.000000
mean        0.048837
std         0.061489
min         0.000000
25%         0.000000
50%         0.028291
75%         0.068710
max         0.415079
Name: similarity_score, dtype: float64


In [34]:
# Set threshold to 0.2 instead of 0.5
df['label'] = (df['similarity_score'] > 0.2).astype(int)

# Now check distribution
print(df['label'].value_counts())


label
0    9139
1     405
Name: count, dtype: int64


In [35]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, df['label'], test_size=0.2, random_state=42, stratify=df['label']
)


In [36]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)


In [37]:
from sklearn.metrics import classification_report, confusion_matrix

print("Logistic Regression Report:")
print(classification_report(y_test, y_pred))

Logistic Regression Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1828
           1       0.95      0.77      0.85        81

    accuracy                           0.99      1909
   macro avg       0.97      0.88      0.92      1909
weighted avg       0.99      0.99      0.99      1909



In [38]:
from sklearn.svm import SVC

svm = SVC()
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)

In [39]:
print("SVM Report:")
print(classification_report(y_test, y_pred_svm))

SVM Report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      1828
           1       0.93      0.85      0.89        81

    accuracy                           0.99      1909
   macro avg       0.96      0.92      0.94      1909
weighted avg       0.99      0.99      0.99      1909



In [42]:
clf_tfidf = LogisticRegression()
clf_tfidf.fit(X_train, y_train)

LogisticRegression()

In [45]:
y_pred = clf_tfidf.predict(X_test)

# Classification Report
print("\nTF-IDF Classification Report:\n")
print(classification_report(y_test, y_pred))


TF-IDF Classification Report:

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1828
           1       0.95      0.77      0.85        81

    accuracy                           0.99      1909
   macro avg       0.97      0.88      0.92      1909
weighted avg       0.99      0.99      0.99      1909

